In [16]:
import pandas as pd
import numpy as np
import re

In [17]:
pd.options.display.max_columns = None

## Cargamos los datos

In [18]:
df = pd.read_csv('../datos/datos_limpios_juntados.csv', index_col = 0)
df.head(2)

,age,gender,country,continent,job_title,work_activities,size_DA_dept,dept size,years_programming,programming_experience,primary_data_tool,first_language_rec,dev_language,IDE,big_data,BI_tools,visualisation,notebooks,sharing,ML,ML_experience
0,50-59,Man,India,Asia & Oceania,Other,None of these activities are an important par...,3-4,small,5-10 years,Senior,"Local development environments (RStudio, Jupyt...",Python,"Python, R",Vim / Emacs,PostgreSQL,None,"Matplotlib ,Seaborn ,Ggplot / ggplot2 ,Shiny ...",Colab Notebooks,"GitHub , Kaggle",5-10 years,5+ years
1,50-59,Man,Indonesia,Asia & Oceania,Program/Project Manager,Build and/or run the data infrastructure that...,1-2,small,20+ years,Senior,"Advanced statistical software (SPSS, SAS, etc.)",Python,"SQL, C, C++, Java","Notepad++,Jupyter Notebook",NaN,NaN,Matplotlib,"Kaggle Notebooks,Colab Notebooks",NaN,< 1 year,< 2 years


In [19]:
df_da_ba = df[(df["job_title"] == "Business Analyst") | (df["job_title"] == "Data Analyst")]
df_da_ba.head(2)

,age,gender,country,continent,job_title,work_activities,size_DA_dept,dept size,years_programming,programming_experience,primary_data_tool,first_language_rec,dev_language,IDE,big_data,BI_tools,visualisation,notebooks,sharing,ML,ML_experience
16,50-59,Man,Belgium,Europe,Data Analyst,Analyze and understand data to influence produ...,5-9,medium,20+ years,Senior,"Local development environments (RStudio, Jupyt...",Python,"Python, SQL","Jupyter (JupyterLab; Jupyter Notebooks; etc) ,...",NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express","Kaggle Notebooks,Colab Notebooks, Databricks ...",NaN,1-2 years,< 2 years
32,18-29,Nonbinary,United States of America,America,Data Analyst,NaN,3-4,small,< 1 years,Junior,NaN,R,R,RStudio,NaN,NaN,Ggplot / ggplot2,Google Cloud Datalab,NaN,I do not use machine learning methods,None


In [20]:
df_da_ba.shape

(3269, 21)

## Exploracion Inicial 

In [21]:
def explore(dataframe, nombre):
    print(f"EXPLORACIÓN DEL DATAFRAME {nombre.upper()}")
    print("--------------")
    print(f"El dataframe {nombre} tiene {dataframe.shape[0]} filas y {dataframe.shape[1]} columnas")
    print("--------------")
    print(f"El dataframe {nombre} tiene las siguientes columnas:")
    for col in dataframe.columns:
        print(col)     
    print("--------------")
    print("Una muestra de dos filas seleccionadas al azar:")
    display(dataframe.sample(2))
    print("--------------")
    print(f"Los principales estadísticos de las variables categóricas son:")
    display(dataframe.describe(include=object).T)
    print("--------------")
    print("El porcentaje de nulos por columna:")
    for i, col in enumerate(dataframe.isnull().sum()):
        print(f"{dataframe.isnull().sum().index[i]}: {col/dataframe.shape[0]*100}")
    print("--------------")


In [22]:
explore(df, "Todos los datos")

EXPLORACIÓN DEL DATAFRAME TODOS LOS DATOS
--------------
El dataframe Todos los datos tiene 25973 filas y 21 columnas
--------------
El dataframe Todos los datos tiene las siguientes columnas:
age
gender
country
continent
job_title
work_activities
size_DA_dept
dept size
years_programming
programming_experience
primary_data_tool
first_language_rec
dev_language
IDE
big_data
BI_tools
visualisation
notebooks
sharing
ML
ML_experience
--------------
Una muestra de dos filas seleccionadas al azar:


,age,gender,country,continent,job_title,work_activities,size_DA_dept,dept size,years_programming,programming_experience,primary_data_tool,first_language_rec,dev_language,IDE,big_data,BI_tools,visualisation,notebooks,sharing,ML,ML_experience
7432,30-39,Man,Japan,Asia & Oceania,Developer Relations/Advocacy,Build and/or run the data infrastructure that...,1-2,small,1-3 years,Junior,"Basic statistical software (Microsoft Excel, G...",Python,"Python, Other","Spyder,Jupyter Notebook",None,Tableau,"Matplotlib ,Seaborn",Colab Notebooks,I do not share my work publicly,1-2 years,< 2 years
6695,30-39,Man,Singapore,Asia & Oceania,Software Engineer,None of these activities are an important par...,0,small,10-20 years,Senior,"Local development environments (RStudio, Jupyt...",Python,"Python, SQL","Jupyter (JupyterLab; Jupyter Notebooks; etc) ,...",NaN,NaN,"Matplotlib ,Plotly / Plotly Express",None,NaN,1-2 years,< 2 years


--------------
Los principales estadísticos de las variables categóricas son:


,count,unique,top,freq
age,25973,5,18-29,14526
gender,25973,5,Man,20598
country,25973,66,India,7434
continent,25973,5,Asia & Oceania,14302
job_title,25973,15,Student,6804
work_activities,15635,96,Analyze and understand data to influence produ...,2891
size_DA_dept,16126,7,1-2,3642
dept size,16126,3,small,9264
years_programming,25973,7,1-3 years,7874
programming_experience,25973,3,Junior,13755


--------------
El porcentaje de nulos por columna:
age: 0.0
gender: 0.0
country: 0.0
continent: 0.0
job_title: 0.0
work_activities: 39.80287221345243
size_DA_dept: 37.9124475416779
dept size: 37.9124475416779
years_programming: 0.0
programming_experience: 0.0
primary_data_tool: 28.81453817425788
first_language_rec: 3.9772070996804376
dev_language: 3.9733569475994304
IDE: 4.000308012166481
big_data: 63.48515766372772
BI_tools: 63.81242059061333
visualisation: 7.711854618257422
notebooks: 4.477726870211373
sharing: 64.53624918184269
ML: 8.543487467754977
ML_experience: 8.543487467754977
--------------


In [23]:
explore(df_da_ba, "Data Analyst y Business Analyst")

EXPLORACIÓN DEL DATAFRAME DATA ANALYST Y BUSINESS ANALYST
--------------
El dataframe Data Analyst y Business Analyst tiene 3269 filas y 21 columnas
--------------
El dataframe Data Analyst y Business Analyst tiene las siguientes columnas:
age
gender
country
continent
job_title
work_activities
size_DA_dept
dept size
years_programming
programming_experience
primary_data_tool
first_language_rec
dev_language
IDE
big_data
BI_tools
visualisation
notebooks
sharing
ML
ML_experience
--------------
Una muestra de dos filas seleccionadas al azar:


,age,gender,country,continent,job_title,work_activities,size_DA_dept,dept size,years_programming,programming_experience,primary_data_tool,first_language_rec,dev_language,IDE,big_data,BI_tools,visualisation,notebooks,sharing,ML,ML_experience
24735,30-39,Man,Other,Other,Data Analyst,Analyze and understand data to influence produ...,5-9,medium,1-3 years,Junior,"Local development environments (RStudio, Jupyt...",Python,"Python, SQL, C++, Javascript, Swift, Bash, MATLAB","Visual Studio ,MATLAB ,Jupyter Notebook","MySQL , PostgreSQL , SQLite , Microsoft SQL Se...","Google Data Studio, Microsoft Azure Synapse","Matplotlib ,Seaborn","Kaggle Notebooks,Azure Notebooks,Google Cloud...","GitHub , Kaggle",1-2 years,< 2 years
8978,30-39,Woman,Colombia,America,Data Analyst,Build and/or run the data infrastructure that...,3-4,small,3-5 years,Senior,"Advanced statistical software (SPSS, SAS, etc.)",Python,"Python, R, SQL","Jupyter (JupyterLab; Jupyter Notebooks; etc) ,...","MongoDB , Microsoft SQL Server",Microsoft Power BI,"Matplotlib ,Seaborn ,Plotly / Plotly Express ...",Google Cloud Notebooks (AI Platform / Vertex A...,"GitHub , Colab",1-2 years,< 2 years


--------------
Los principales estadísticos de las variables categóricas son:


,count,unique,top,freq
age,3269,5,18-29,1562
gender,3269,5,Man,2538
country,3269,66,India,795
continent,3269,5,Asia & Oceania,1635
job_title,3269,2,Data Analyst,2301
work_activities,2971,73,Analyze and understand data to influence produ...,1117
size_DA_dept,3064,7,1-2,743
dept size,3064,3,small,1778
years_programming,3269,7,1-3 years,991
programming_experience,3269,3,Junior,1881


--------------
El porcentaje de nulos por columna:
age: 0.0
gender: 0.0
country: 0.0
continent: 0.0
job_title: 0.0
work_activities: 9.115937595594984
size_DA_dept: 6.271030896298563
dept size: 6.271030896298563
years_programming: 0.0
programming_experience: 0.0
primary_data_tool: 23.248699908228815
first_language_rec: 8.442948914040992
dev_language: 8.4123585194249
IDE: 8.504129703273172
big_data: 47.62924441725298
BI_tools: 47.96573875802998
visualisation: 10.55368614255124
notebooks: 8.748852860201897
sharing: 49.25053533190578
ML: 11.134903640256958
ML_experience: 11.134903640256958
--------------


## Extraemos los valores únicos de cada columna

In [24]:
#esta función crea un dicionario con el número de la columna como keys y una lista de los valores únicos de la columna como values
def dict_respuestas(dataframe):
    
    dic_respuestas = {}
    
    for col in dataframe.columns:
        lista_nueva = []
        #ignoramos la columna index
        if col == "index":
            pass
        #comprobamos si los valores únicos son más de 20 - si es una pregunta multirespuesta 
        elif len(dataframe[col].unique()) > 22:
            #crea una lista de los valores únicos
                lista_unicos = list(dataframe[col].unique())
                for sublist in lista_unicos:
                    #cada lista dentro de los valores únicos se divide por las comas 
                    try:
                        lista_nueva.extend(sublist.split(","))
                    except:
                        pass
                    #lista de todas las posibles respuestas individuales
                    #la convertimos en set para eliminar los duplicados 
                    set_unicos = set(lista_nueva)
                    #lo volvemos a convertir en lista para poder usarla cómodamente
                    lista_nueva = list(set_unicos)
        else:
            #si no es multirespuesta podemos usar el unique para ver los valores únicos 
            lista_nueva = list(dataframe[col].unique())
        #list comprehension para quitar los nulos    
        lista_sin_nan = [item for item in lista_nueva if not (pd.isnull(item)) == True]
        #metemos la lista sin nulos en un diccionario donde el key es el número de la columna y los values con la lista de valores únicos lista_sin_nan
        dic_respuestas[col] = lista_sin_nan
        
        print("-------------------")
        print (f"{col} : {dic_respuestas[col]}")

    return dic_respuestas


In [25]:
diccionario_respuestas = dict_respuestas(df_da_ba)

-------------------
age : ['50-59', '18-29', '30-39', '40-49', '60+']
-------------------
gender : ['Man', 'Nonbinary', 'Woman', 'Prefer not to say', 'Prefer to self-describe']
-------------------
country : ['South Africa', 'Algeria', 'Ethiopia', 'Austria', 'Norway', 'Kenya', 'Romania', 'Turkey', 'Indonesia', 'Russia', 'Nigeria', 'United Arab Emirates', 'Iraq', 'Mexico', 'Malaysia', 'Ireland', 'Switzerland', 'Pakistan', 'Spain', 'Sweden', 'Iran', 'Argentina', 'Italy', 'Egypt', 'Belgium', 'Poland', 'South Korea', 'Singapore', 'France', 'Sri Lanka', 'Belarus', 'Germany', 'Netherlands', 'Ukraine', 'Morocco', 'United Kingdom of Great Britain and Northern Ireland', 'Israel', 'Viet Nam', 'Portugal', 'Chile', 'India', 'Japan', 'Philippines', 'Tunisia', 'I do not wish to disclose my location', 'Colombia', 'Canada', 'Australia', 'Brazil', 'Denmark', 'Ecuador', 'Hong Kong (S.A.R.)', 'Taiwan', ' Islamic Republic of...', 'Ghana', 'Saudi Arabia', 'Bangladesh', 'Kazakhstan', 'Other', 'Peru', 'Nepal'

## Contamos los valores y calculamos el %

In [26]:
#devuelve el valor de cada valor único y su porcentaje
#el diccionario abajo es el diccionario de respuestas que acabamos de hacer 
def count_respuestas(dataframe, **diccionario):
    for key in diccionario:
        #usamos el diccionario, sacamos los valores únicos de cada pregunta para encontrar la columna del dataframe, 
        # y el sum nos dice cuántas veces aparece      
        if key == "index":
            pass
        elif len(dataframe[key].unique()) > 22:
            print("----------------")
            print(f"PREGUNTA MULTIRESPUESTA {key}")  
            for respuesta in diccionario[key]:
                try:
                    total = dataframe[key].str.contains(respuesta, regex = True).sum()
                    print(f"{respuesta}: total: {total}, {round(total/dataframe.shape[0]*100, 1)}%")
                except:
                    print(f"no se puede para {respuesta}")
        else:
            print("------------")
            print(f"PREGUNTA {key} (%)")
            print(f"{round((dataframe[key].value_counts()/dataframe.shape[0]*100),2)}")


In [27]:
count_respuestas(df_da_ba, **diccionario_respuestas)

------------
PREGUNTA age (%)
18-29    47.78
30-39    30.74
40-49    13.67
50-59     5.51
60+       2.29
Name: age, dtype: float64
------------
PREGUNTA gender (%)
Man                        77.64
Woman                      20.80
Prefer not to say           1.16
Nonbinary                   0.24
Prefer to self-describe     0.15
Name: gender, dtype: float64
----------------
PREGUNTA MULTIRESPUESTA country
South Africa: total: 11, 0.3%
Algeria: total: 2, 0.1%
Ethiopia: total: 2, 0.1%
Austria: total: 5, 0.2%
Norway: total: 3, 0.1%
Kenya: total: 48, 1.5%
Romania: total: 4, 0.1%
Turkey: total: 36, 1.1%
Indonesia: total: 60, 1.8%
Russia: total: 108, 3.3%
Nigeria: total: 132, 4.0%
United Arab Emirates: total: 17, 0.5%
Iraq: total: 4, 0.1%
Mexico: total: 49, 1.5%
Malaysia: total: 17, 0.5%
Ireland: total: 86, 2.6%
Switzerland: total: 8, 0.2%
Pakistan: total: 50, 1.5%
Spain: total: 48, 1.5%
Sweden: total: 10, 0.3%
Iran: total: 16, 0.5%
Argentina: total: 32, 1.0%
Italy: total: 31, 0.9%
Egypt: tota

/tmp/ipykernel_1251/2681907443.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  total = dataframe[key].str.contains(respuesta, regex = True).sum()
/tmp/ipykernel_1251/2681907443.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  total = dataframe[key].str.contains(respuesta, regex = True).sum()



Leaflet / Folium : total: 94, 2.9%
Ggplot / ggplot2 : total: 830, 25.4%
Seaborn : total: 1559, 47.7%
 Matplotlib : total: 1984, 60.7%
Plotly / Plotly Express : total: 796, 24.3%
----------------
PREGUNTA MULTIRESPUESTA notebooks
 Databricks Collaborative Notebooks : total: 88, 2.7%
Other: total: 113, 3.5%
None: total: 933, 28.5%
 Amazon EMR Notebooks : total: 45, 1.4%
Azure Notebooks: total: 152, 4.6%
 Paperspace / Gradient : total: 23, 0.7%
 Amazon Sagemaker Studio Notebooks : total: 73, 2.2%
Colab Notebooks: total: 920, 28.1%
 IBM Watson Studio : total: 144, 4.4%
 Binder / JupyterHub : total: 235, 7.2%
Google Cloud Notebooks (AI Platform / Vertex AI) : total: 0, 0.0%
 Deepnote Notebooks : total: 26, 0.8%
 Zeppelin / Zepl Notebooks : total: 28, 0.9%
 Code Ocean : total: 22, 0.7%
 Kaggle Notebooks: total: 1077, 32.9%
Google Cloud Datalab: total: 274, 8.4%
 Observable Notebooks : total: 31, 0.9%
----------------
PREGUNTA MULTIRESPUESTA sharing
Other: total: 56, 1.7%
 Colab : total: 275

In [ ]:
df_crosstab_gen_exp = pd.crosstab(df_da_ba['gender'], df_da_ba['experiencia programacion'], normalize = True) * 100
df_crosstab_gen_exp

experiencia programacion,Junior,Senior,Sin experiencia
gender,,,
Man,44.141939,27.256042,6.240441
Nonbinary,0.122362,0.091771,0.030590
Prefer not to say,0.550627,0.520037,0.091771
Prefer to self-describe,0.061181,0.061181,0.030590
Woman,12.664423,6.118079,2.018966


In [ ]:
# Sacamos tabla de contigencia cruzando género y la pregunta q8, donde se recoge la información de recomendación sobre herramientas de programación para iniciarse en análisis de datos

df_crosstab_gen_herr = pd.crosstab(df_da_ba['gender'], df_da_ba['Q8'], normalize = True, margins = True) * 100
df_crosstab_gen_herr.T

gender,Man,Nonbinary,Prefer not to say,Prefer to self-describe,Woman,All
Q8,,,,,,
Bash,0.133645,0.000000,0.033411,0.000000,0.033411,0.200468
C,0.601403,0.000000,0.000000,0.000000,0.233879,0.835282
C++,1.002339,0.000000,0.033411,0.000000,0.267290,1.303040
Java,0.701637,0.000000,0.000000,0.000000,0.300702,1.002339
Javascript,0.334113,0.000000,0.066823,0.000000,0.033411,0.434347
Julia,0.367524,0.033411,0.000000,0.000000,0.066823,0.467758
MATLAB,0.434347,0.000000,0.000000,0.000000,0.233879,0.668226
None,0.367524,0.000000,0.033411,0.000000,0.066823,0.467758
Other,0.735048,0.000000,0.000000,0.000000,0.066823,0.801871
